## Import Dependencies

In [1]:
import json
import os
from glob import glob
import shutil

## Mount Google Drive

In [2]:
from google.colab import drive

drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [3]:
INPUT_JSON_DIR = "/content/drive/My Drive/Red Cross Project Materials/reorganized_cityscapes/gtFine"
OUTPUT_LABEL_DIR = "/content/drive/My Drive/Red Cross Project Materials/for_finetuning"

os.makedirs(OUTPUT_LABEL_DIR, exist_ok=True)

## JSON --> YOLO Pipeline

In [4]:
def JSON_to_YOLO(test_train, is_buildling):
  """
  Converts a FOLDER of json files to the corresponding YOLO.txt labels,
  including buildings only.

  Key arguments correspond to the naming of folders
  test_train: strings "train" or "val"
  is_buildling: strings "building" or "non_buildling"
  """

  # Make train/val directory
  train_val = os.path.join(OUTPUT_LABEL_DIR, test_train)
  os.makedirs(train_val, exist_ok=True)

  # Get all the .json files
  json_files = glob(os.path.join(INPUT_JSON_DIR, test_train, is_buildling, "*.json"))

  for jf in json_files:
    with open(jf, "r") as f:
      data = json.load(f)

    W = data["imgWidth"]
    H = data["imgHeight"]
    objects = data["objects"]

    # Image filename ex) aachen_000000_000019_leftImg8bit
    # JSON filname ex) aachen_000000_000019_gtFine_polygons

    base = os.path.splitext(os.path.basename(jf))[0]
    out_path = os.path.join(OUTPUT_LABEL_DIR, test_train, "labels", base[:-16] +"_leftImg8bit.txt") # removes the building, non_buildling subfolders

    lines = [] # to store the bounding box info for each instance of a building
    for obj in objects:
      label = obj["label"]
      if label != "building":
        continue  # skip obj

      # else...
      x_coords = []
      y_coords = []
      polygon = obj["polygon"] # nested list

      for x, y in polygon:
        x_coords.append(x)
        y_coords.append(y)

      y_min = min(y_coords)
      y_max = max(y_coords)
      x_min = min(x_coords)
      x_max = max(x_coords)

      # Get bounding box info (normalized)
      x_center = ((x_min + x_max) / 2) / W
      y_center = ((y_min + y_max) / 2) / H
      width = (x_max - x_min) / W
      height = (y_max - y_min) /H

      # Format = class_id, normalized x_coord, normalized y_coord, normalized wigth, normalized height
      line = "0 " + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height) # hardcoding building class id as 0
      lines.append(line)

    # Write YOLO txt file
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w") as f:
      f.write("\n".join(lines))

In [5]:
for train_val in os.listdir(INPUT_JSON_DIR):
  if train_val == "test":
    continue # skip, inaccurate gt labels in test

  train_val_dir = os.path.join(INPUT_JSON_DIR, train_val)

  for is_building in os.listdir(train_val_dir):
    JSON_to_YOLO(train_val, is_building)

## Reorganize Drive

In [11]:
INPUT_IMG_DIR = "/content/drive/My Drive/Red Cross Project Materials/processed_data"
OUTPUT_IMG_DIR = "/content/drive/My Drive/Red Cross Project Materials/for_finetuning"

# test, val, train folders
for train_val_test in os.listdir(INPUT_IMG_DIR):
  output_tvt = os.path.join(OUTPUT_IMG_DIR, train_val_test)
  os.makedirs(output_tvt, exist_ok=True)

  # building, non_building folders
  train_val_test_dir = os.path.join(INPUT_IMG_DIR, train_val_test)
  for is_building in os.listdir(train_val_test_dir):

    # Actual images
    image_dir = os.path.join(train_val_test_dir, is_building)
    for img in os.listdir(image_dir):
      src_file = os.path.join(image_dir, img)
      dst_file = os.path.join(OUTPUT_IMG_DIR, train_val_test, "images", img)
      os.makedirs(os.path.dirname(dst_file), exist_ok=True)
      if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)

## Finetune YOLO

In [8]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.0 MB/s eta 0:00:00


In [12]:
yaml_text = """
train: /content/drive/My Drive/Red Cross Project Materials/for_finetuning/train
val: /content/drive/My Drive/Red Cross Project Materials/for_finetuning/val

nc: 1
names: ["building"]
"""

with open("/content/drive/My Drive/Red Cross Project Materials/for_finetuning/data.yaml", "w") as f:
    f.write(yaml_text)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # nano
model.train(
    data="/content/drive/My Drive/Red Cross Project Materials/for_finetuning/data.yaml",
    epochs=100,
    imgsz=512, # downsize image...
    batch=8,
    augment=False,
    project="/content/drive/My Drive/Red Cross Project Materials/saved_models",
    name="yolov8n_building_detector"
)

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/My Drive/Red Cross Project Materials/for_finetuning/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_building_detector3, nbs=64, nms=False, opset=None, optimize=False, optimizer=aut

## Extracting Bounding Boxes

In [ ]:
import os
import glob
import cv2
from ultralytics import YOLO

In [ ]:
# ------------------------------------------------------
# SETTINGS
# ------------------------------------------------------
MODEL_PATH = "best.pt"                # your trained YOLO model
INPUT_FOLDER = "images/"              # folder containing input images
OUTPUT_FOLDER = "extracted_buildings" # where cropped buildings will be saved
TARGET_CLASS = 0                      # class ID for "building"
# ------------------------------------------------------

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Load YOLO model
model = YOLO(MODEL_PATH)

# Collect all images
image_paths = glob(os.path.join(INPUT_FOLDER, "*.png"))

In [ ]:
# Loop through images
for img in image_paths:
  print(f"Processing: {img}")
  img_name = os.path.splitext(os.path.basename(img))[0]

  # Run inference
  results = model(img)[0]

  # Load original image
  img = cv2.imread(img)

  crop_count = 0

  # Loop through each detection
  for box in results.boxes:
    cls = int(box.cls[0])
    if cls != TARGET_CLASS:
      continue  # skip other classes

    # Get box coordinates
    x1, y1, x2, y2 = map(int, box.xyxy[0])

    # Crop the region
    crop = img[y1:y2, x1:x2]
    if crop.size == 0:
      continue  # skip invalid crops

    # Save crop
    save_path = os.path.join(
      OUTPUT_FOLDER, f"{img_name}_building_{crop_count}.png"
    )
    cv2.imwrite(save_path, crop)
    crop_count += 1

  print(f"  Saved {crop_count} building crops.")